In [2]:
import pandas as pd
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
csv_file = pd.read_csv(f"{location}/ERC_correlation.csv")

csv_file.index = csv_file.columns
string_to_remove = "_nostop.fas.aligned.fas.best"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)

string_to_remove = ".fas.ClipKIT"
csv_file.columns = csv_file.columns.str.replace(string_to_remove, "", regex=False)
csv_file.index = csv_file.index.str.replace(string_to_remove, "", regex=False)


matrix_long = csv_file.reset_index().melt(
    id_vars="index",         # Keeps row names as a column
    var_name="Gene1",       # New name for columns
    value_name="Gene2"       # New name for matrix values
)

# Rename 'index' to 'Row' for clarity
diapause = matrix_long.rename(columns={"index": "Row"})

location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.ERC_new_diapause_nondia/3.For Clusturing"

# diapause = pd.read_csv(f"{location}/ERC_correlation_diapause_long.csv")
# location = "/mnt/c/Users/Saurav Baral/Desktop/20.RERConverge_trimmed/unTrimmed/3.For_tree"
gene_anno_dic = {}
with open(f"{location}/correlation_2_with_annotation.csv", 'r') as cor_file:
    for lines in cor_file.readlines()[1:]:
        gene_name = lines.split(",")[0].split("_")[0][1:]
        anno_name = lines.split(",")[-1].replace('"', '')
        gene_anno_dic[gene_name]= anno_name.strip()


diapause_rn = diapause.rename(columns={"Gene2": "Correlation"})
diapause_rn = diapause_rn.rename(columns={"Gene1": "Gene2"})
diapause_rn = diapause_rn.rename(columns={"Row": "Gene1"})

def check_annotation(name):
    return gene_anno_dic.get(name, name)

diapause_rn["Annotation_gene1"] = diapause_rn["Gene1"].apply(check_annotation)
diapause_rn["Annotation_gene2"] = diapause_rn["Gene2"].apply(check_annotation)

diapause_rn["Correlation"].fillna(1, inplace=True)

# print(diapause_rn)

genes = diapause_rn["Gene1"].unique()
# df_filtered = diapause_rn[diapause_rn["Correlation"] >= 0.1]
# diapause_rn["Correlation"] += 1
df_filtered = diapause_rn
gene_dictionary = {}
to_remove = ["tim","9062at7088","per","TIMELESS","CLOCK"]
for row in df_filtered.itertuples():
    if (row.Annotation_gene1 not in to_remove) and (row.Annotation_gene2 not in to_remove):
        if row.Annotation_gene1 not in gene_dictionary:
            gene_dictionary[row.Annotation_gene1] = {}
        if row.Annotation_gene1 != row.Annotation_gene2:
            gene_dictionary[row.Annotation_gene1][row.Annotation_gene2] = row.Correlation

gene_dict = gene_dictionary
print("done")


done


In [3]:
import networkx as nx
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform

genes = list(gene_dict.keys())

# Initialize a similarity matrix
n = len(genes)
similarity_matrix = np.zeros((n, n))



for i, gene1 in enumerate(genes):
    for j, gene2 in enumerate(genes):
        if gene2 in gene_dict[gene1]:
            similarity_matrix[i, j] = round(gene_dict[gene1][gene2],2)
            similarity_matrix[j, i] = round(gene_dict[gene1][gene2],2)  # Ensure symmetry
        else:
            similarity_matrix[i, j] = 0  # No direct similarity (or self-similarity)
            similarity_matrix[j, i] = 0


min_similarity = np.min(similarity_matrix)
print(min_similarity)

# Convert similarity matrix to a distance matrix
distance_matrix = 1 - similarity_matrix

np.fill_diagonal(distance_matrix, 0)

# Flatten the distance matrix to a condensed form (required by scipy)
condensed_distance_matrix = squareform(distance_matrix)


# print(similarity_matrix)
# print(distance_matrix)

-0.58


In [4]:
def average_similarity(cluster, similarity_matrix, genes):
    """
    Compute the average similarity within a cluster.
    """
    if len(cluster) < 2:
        return 0.0  # No similarity for single-gene clusters
    
    # Get indices of genes in the cluster
    indices = [genes.index(gene) for gene in cluster]
    
    # Extract the submatrix for the cluster
    submatrix = similarity_matrix[np.ix_(indices, indices)]
    
    # Compute the average similarity (excluding diagonal)
    return np.sum(submatrix) / (len(cluster) * (len(cluster) - 1))

import statistics

import matplotlib.pyplot as plt


def get_clusters(i):
    n_clusters = i # Adjust this based on your needs
    clustering = AgglomerativeClustering(
        n_clusters=n_clusters,
        affinity="precomputed",  # Use precomputed distances
        linkage="average"  # You can change this to "single", "complete", or "ward"
    )
    
    # Fit the model
    clustering.fit(distance_matrix)
    
    # Get the cluster labels
    labels = clustering.labels_
    
    # Organize genes into clusters
    clusters = {}
    for gene, label in zip(genes, labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(gene)
    return(clusters)
    # Print the clusters
    

In [12]:
number_of_cluster = []
mean_cluster_size = []
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
# for i in range(500,3000,50):
i = 2450
clusters = get_clusters(i)
similarity_list = []
cluster_size_list = []
output = "Cluster_id\tGenes\tAverage_similarity"
for cluster_id, genes_in_cluster in clusters.items():

    
    if len(genes_in_cluster) > 2 and round(average_similarity(genes_in_cluster, similarity_matrix, genes),2) >0.40: 
        # print(cluster_id, genes_in_cluster)
        output += f"\n{cluster_id}\t{','.join(genes_in_cluster)}\t{round(average_similarity(genes_in_cluster, similarity_matrix, genes),2)}"
        # print(output)
        # break
        
        
        #     similarity_list.append(average_similarity(genes_in_cluster, similarity_matrix, genes))
    #     cluster_size_list.append(len(genes_in_cluster))
    # # if "period" in genes_in_cluster and len(genes_in_cluster) <= 5 :
    # #     print(f"Cluster {cluster_id}: {genes_in_cluster}, {average_similarity(genes_in_cluster, similarity_matrix, genes)}")
    # # if "cycle" in genes_in_cluster:
    # #         print(f"Cluster {cluster_id}: {genes_in_cluster}, {average_similarity(genes_in_cluster, similarity_matrix, genes)}")


        
        
# number_of_cluster.append(len(cluster_size_list))
# mean_cluster_size.append(statistics.mean(cluster_size_list))
    # break
# print(number_of_cluster,mean_cluster_size)
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.ERC_new_diapause_nondia/3.For Clusturing"

with open(f"{location}/clusters.txt", 'w') as out_file:
    out_file.write(output)

In [19]:
import scipy.cluster.hierarchy as sch
Z = sch.linkage(condensed_distance_matrix, method="average")

In [23]:
from pycirclize import Circos
# from pycirclize.parser.tree import Tree


import scipy.cluster.hierarchy as sch
# Convert linkage matrix to Newick format for pycirclize
newick_str = sch.to_tree(Z, rd=True).to_newick(labels=genes)

# Create a Circos plot
circos = Circos.initialize_from_tree(newick_str, tree_params={"leaf_label_size": 10})
circos.plotfig()

AttributeError: 'tuple' object has no attribute 'to_newick'

In [45]:
import ete3
from ete3 import Tree
import scipy.cluster.hierarchy as sch

def safe_name(name):
    """ Ensure names are quoted if needed """
    if any(c in name for c in "(),:;"):
        return f"'{name}'"  # Quote names if they contain special characters
    return name

def linkage_to_newick(Z, labels):
    """ Convert linkage matrix to Newick format """
    tree, _ = sch.to_tree(Z, rd=True)  # Extract the tree object

    def build_newick(node):
        if node is None:
            return ""

        if node.left is None and node.right is None:
            return f"{safe_name(labels[node.id])}:{max(node.dist, 0.001):.5f}"  # Avoid zero distance

        left_subtree = build_newick(node.left)
        right_subtree = build_newick(node.right)
        return f"({left_subtree},{right_subtree}):{max(node.dist, 0.001):.5f}"  # Ensure valid distance

    return build_newick(tree) + ";"

# Generate the Newick string
newick_str = linkage_to_newick(Z, genes).replace("'Su(P)'","sup").replace("'su(Hw)'","suhw").replace("'l(3)87Df'","87DF").replace("'mod(mdg4)'",'mgd4')
# print(newick_str)
# Load and display the tree
t = Tree(newick_str, format=1)


# t.show()

ImportError: cannot import name 'TreeStyle' from 'ete3' (/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/ete3/__init__.py)

In [57]:
newick_str = linkage_to_newick(Z, genes).replace("'Su(P)'","sup").replace("'su(Hw)'","suhw").replace("'l(3)87Df'","87DF").replace("'mod(mdg4)'",'mgd4')

location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.ERC_new_diapause_nondia/3.For Clusturing"
with open(f"{location}/tree.tre", 'w') as out_file:
    out_file.write(str(newick_str))

In [48]:

t = Tree(newick_str, format=1)
t.render("tree.svg", w=500, units="px")

ImportError: cannot import name 'TreeStyle' from 'ete3' (/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/ete3/__init__.py)

In [50]:
import matplotlib.pyplot as plt
from Bio import Phylo

# Load the tree from a Newick string
# newick_str = "(('Gene1':0.5,'Gene2':0.6):0.7,('Gene3':0.4,'Gene4':0.8):0.9);"
tree = Phylo.read(newick_str, "newick")

# Set figure size
fig, ax = plt.subplots(figsize=(6, 6))

# Draw the tree in a circular format and save as SVG
Phylo.draw(tree, do_show=False, axes=ax)
plt.savefig("tree.svg", format="svg")
plt.close()



OSError: [Errno 36] File name too long: '((((((VPS33A:0.00100,(GNS:0.00100,4668at7088:0.00100):0.64000):0.69000,((IFT122:0.00100,RILPL1:0.00100):0.63000,(DNA2:0.00100,(SNX2:0.00100,8779at7088:0.00100):0.63000):0.70000):0.74667):0.81733,((((PRPF40A:0.00100,ATG9A:0.00100):0.68000,((UBXN6:0.00100,NUP155:0.00100):0.60000,(SH3GL1:0.00100,RB1CC1:0.00100):0.62000):0.74250):0.77625,((SRRT:0.00100,(KIFAP3:0.00100,VPS8:0.00100):0.65000):0.75500,((PPFIBP2:0.00100,CCDC39:0.00100):0.54000,(2325at7088:0.00100,(CIT:0.00100,REPS1:0.00100):0.60000):0.71000):0.75833):0.81667):0.84187,((((TXNDC11:0.00100,NCSTN:0.00100):0.53000,((GLIS2:0.00100,8186at7088:0.00100):0.58000,(WDR81:0.00100,ATR:0.00100):0.59000):0.67250):0.71875,(((550at7088:0.00100,(1037at7088:0.00100,8408at7088:0.00100):0.59000):0.61000,(RAB11FIP4:0.00100,(1248at7088:0.00100,rom-4:0.00100):0.55000):0.66500):0.69889,(CAP1:0.00100,(1391at7088:0.00100,ABHD2:0.00100):0.57000):0.70500):0.74333):0.80222,((CLASRP:0.00100,UBE3C:0.00100):0.63000,((FN

In [43]:
!conda install -c etetoolkit ete3


Retrieving notices: ...working... done
Channels:
 - etetoolkit
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 24.5.0
    latest version: 25.1.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/saurav/miniconda3/envs/genomics_work

  added / updated specs:
    - ete3


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.13            |       hb9d3cd8_0         547 KB  conda-forge
    attr-2.5.1                 |       h166bdaf_1          69 KB  conda-forge
    c-ares-1.34.4              |       hb9d3cd8_0         201 KB  conda-forge
    ca-certificates-2025.1.31  |       hbcca054_0         154 KB  conda-forge
    cairo-1.18.2               |       h3394656_1         956 KB  conda-forge
    c

In [39]:
import ete3
ete3.__file__

'/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/ete3/__init__.py'

In [42]:
from ete3 import TreeStyle
print(TreeStyle)

ImportError: cannot import name 'TreeStyle' from 'ete3' (/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/ete3/__init__.py)

In [55]:
def linkage_to_newick(Z, labels):
    """ Convert linkage matrix to Newick format """
    tree, _ = sch.to_tree(Z, rd=True)

    def build_newick(node):
        if node.is_leaf():
            return f"{labels[node.id]}:{node.dist:.5f}"
        left = build_newick(node.left)
        right = build_newick(node.right)
        return f"({left},{right}):{node.dist:.5f}"

    return build_newick(tree) + ";"

newick_str = linkage_to_newick(Z, genes)
print(newick_str)

((((((VPS33A:0.00000,(GNS:0.00000,4668at7088:0.00000):0.64000):0.69000,((IFT122:0.00000,RILPL1:0.00000):0.63000,(DNA2:0.00000,(SNX2:0.00000,8779at7088:0.00000):0.63000):0.70000):0.74667):0.81733,((((PRPF40A:0.00000,ATG9A:0.00000):0.68000,((UBXN6:0.00000,NUP155:0.00000):0.60000,(SH3GL1:0.00000,RB1CC1:0.00000):0.62000):0.74250):0.77625,((SRRT:0.00000,(KIFAP3:0.00000,VPS8:0.00000):0.65000):0.75500,((PPFIBP2:0.00000,CCDC39:0.00000):0.54000,(2325at7088:0.00000,(CIT:0.00000,REPS1:0.00000):0.60000):0.71000):0.75833):0.81667):0.84187,((((TXNDC11:0.00000,NCSTN:0.00000):0.53000,((GLIS2:0.00000,8186at7088:0.00000):0.58000,(WDR81:0.00000,ATR:0.00000):0.59000):0.67250):0.71875,(((550at7088:0.00000,(1037at7088:0.00000,8408at7088:0.00000):0.59000):0.61000,(RAB11FIP4:0.00000,(1248at7088:0.00000,rom-4:0.00000):0.55000):0.66500):0.69889,(CAP1:0.00000,(1391at7088:0.00000,ABHD2:0.00000):0.57000):0.70500):0.74333):0.80222,((CLASRP:0.00000,UBE3C:0.00000):0.63000,((FNIP1:0.00000,KIAA1468:0.00000):0.71000,((F

In [1]:
import numpy as np
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform
from collections import defaultdict
import random

similarity_matrix = np.array([
    [1.0, 0.8, 0.6, 0.2, 0.1, 0.4],
    [0.8, 1.0, 0.7, 0.3, 0.2, 0.5],
    [0.6, 0.7, 1.0, 0.4, 0.3, 0.6],
    [0.2, 0.3, 0.4, 1.0, 0.8, 0.7],
    [0.1, 0.2, 0.3, 0.8, 1.0, 0.9],
    [0.4, 0.5, 0.6, 0.7, 0.9, 1.0],
])

genes = ["gene1", "gene2", "gene3", "gene4", "gene5", "gene6"]

# Convert similarity to distance
distance_matrix = 1 - similarity_matrix
distance_matrix = (distance_matrix + distance_matrix.T) / 2  # Ensure symmetry
np.fill_diagonal(distance_matrix, 0)  # Set diagonal to zero
distance_matrix = np.round(distance_matrix, 3)  # Fix floating-point issues
condensed_matrix = squareform(distance_matrix)

# Perform original hierarchical clustering
Z_original = sch.linkage(condensed_matrix, method="average")




num_bootstraps = 100  # Number of bootstrap resamples
bootstrap_support = defaultdict(int)

for _ in range(num_bootstraps):
    # Resample genes with replacement
    resampled_indices = np.random.choice(len(genes), size=len(genes), replace=True)
    resampled_genes = [genes[i] for i in resampled_indices]
    
    # Subset distance matrix
    resampled_distances = distance_matrix[np.ix_(resampled_indices, resampled_indices)]
    
    # Compute linkage on bootstrap sample
    condensed_resampled = squareform(resampled_distances)
    Z_resampled = sch.linkage(condensed_resampled, method="average")
    
    # Convert to set of clusters
    for i in range(len(Z_resampled)):
        cluster = frozenset(resampled_genes[int(Z_resampled[i, 0]) : int(Z_resampled[i, 1]) + 1])
        bootstrap_support[cluster] += 1

# Compute bootstrap percentages
for cluster in bootstrap_support:
    bootstrap_support[cluster] = (bootstrap_support[cluster] / num_bootstraps) * 100

# Print bootstrap confidence values
print("Bootstrap support values for clusters:")
for cluster, support in bootstrap_support.items():
    
    print(f"Cluster {cluster}: {support:.2f}% confidence")

Bootstrap support values for clusters:
Cluster frozenset({'gene6', 'gene3'}): 9.00% confidence
Cluster frozenset({'gene6'}): 8.00% confidence
Cluster frozenset({'gene2', 'gene6', 'gene5', 'gene3'}): 4.00% confidence
Cluster frozenset({'gene6', 'gene2', 'gene3'}): 4.00% confidence
Cluster frozenset(): 119.00% confidence
Cluster frozenset({'gene2', 'gene1'}): 13.00% confidence
Cluster frozenset({'gene4', 'gene5', 'gene3'}): 2.00% confidence
Cluster frozenset({'gene2', 'gene5', 'gene3', 'gene1'}): 8.00% confidence
Cluster frozenset({'gene4'}): 18.00% confidence
Cluster frozenset({'gene4', 'gene3'}): 9.00% confidence
Cluster frozenset({'gene6', 'gene5'}): 20.00% confidence
Cluster frozenset({'gene3'}): 17.00% confidence
Cluster frozenset({'gene6', 'gene4', 'gene2', 'gene1'}): 4.00% confidence
Cluster frozenset({'gene6', 'gene4'}): 8.00% confidence
Cluster frozenset({'gene2', 'gene4', 'gene5', 'gene3'}): 4.00% confidence
Cluster frozenset({'gene4', 'gene2', 'gene3'}): 4.00% confidence
Clust

In [2]:
import numpy as np
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform
from collections import defaultdict
import random

# Example similarity matrix (should be replaced with actual data)
similarity_matrix = np.array([
    [1.0, 0.8, 0.6, 0.2, 0.1, 0.4],
    [0.8, 1.0, 0.7, 0.3, 0.2, 0.5],
    [0.6, 0.7, 1.0, 0.4, 0.3, 0.6],
    [0.2, 0.3, 0.4, 1.0, 0.8, 0.7],
    [0.1, 0.2, 0.3, 0.8, 1.0, 0.9],
    [0.4, 0.5, 0.6, 0.7, 0.9, 1.0],
])

genes = ["gene1", "gene2", "gene3", "gene4", "gene5", "gene6"]

# Convert similarity to distance
distance_matrix = 1 - similarity_matrix
distance_matrix = (distance_matrix + distance_matrix.T) / 2  # Ensure symmetry
np.fill_diagonal(distance_matrix, 0)  # Set diagonal to zero
distance_matrix = np.round(distance_matrix, 3)  # Fix floating-point issues
condensed_matrix = squareform(distance_matrix)

# Perform original hierarchical clustering
Z_original = sch.linkage(condensed_matrix, method="average")

# Bootstrapping
num_bootstraps = 100  # Number of bootstrap resamples
bootstrap_support = defaultdict(int)

for _ in range(num_bootstraps):
    # Resample genes with replacement
    resampled_indices = np.random.choice(len(genes), size=len(genes), replace=True)
    resampled_genes = [genes[i] for i in resampled_indices]
    
    # Subset distance matrix
    resampled_distances = distance_matrix[np.ix_(resampled_indices, resampled_indices)]
    
    # Compute linkage on bootstrap sample
    condensed_resampled = squareform(resampled_distances)
    Z_resampled = sch.linkage(condensed_resampled, method="average")
    
    # Convert to set of clusters
    for i in range(len(Z_resampled)):
        cluster = frozenset(resampled_genes[int(Z_resampled[i, 0]) : int(Z_resampled[i, 1]) + 1])
        bootstrap_support[cluster] += 1

# Compute bootstrap percentages
for cluster in bootstrap_support:
    bootstrap_support[cluster] = (bootstrap_support[cluster] / num_bootstraps) * 100

# Function to create Newick format with bootstrap support
def generate_newick(Z, bootstrap_support, genes):
    def recurse(node):
        if node < len(genes):  # Leaf node
            return genes[int(node)]
        else:  # Internal node
            left = recurse(Z[node - len(genes), 0])
            right = recurse(Z[node - len(genes), 1])
            support = bootstrap_support.get(frozenset([left, right]), 0)
            return f"({left},{right}){':' + str(round(support, 2)) if support > 0 else ''}"

    # Start the recursive generation of the Newick string
    return recurse(len(Z) + len(genes) - 1) + ';'

# Generate Newick tree with bootstrap support
newick_tree = generate_newick(Z_original, bootstrap_support, genes)

# Print the Newick tree with bootstrap values
print("Newick Tree with Bootstrap Confidence:")
print(newick_tree)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [3]:
import numpy as np
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform
from collections import defaultdict
import random

# Example similarity matrix (should be replaced with actual data)
similarity_matrix = np.array([
    [1.0, 0.8, 0.6, 0.2, 0.1, 0.4],
    [0.8, 1.0, 0.7, 0.3, 0.2, 0.5],
    [0.6, 0.7, 1.0, 0.4, 0.3, 0.6],
    [0.2, 0.3, 0.4, 1.0, 0.8, 0.7],
    [0.1, 0.2, 0.3, 0.8, 1.0, 0.9],
    [0.4, 0.5, 0.6, 0.7, 0.9, 1.0],
])

genes = ["gene1", "gene2", "gene3", "gene4", "gene5", "gene6"]

# Convert similarity to distance
distance_matrix = 1 - similarity_matrix
distance_matrix = (distance_matrix + distance_matrix.T) / 2  # Ensure symmetry
np.fill_diagonal(distance_matrix, 0)  # Set diagonal to zero
distance_matrix = np.round(distance_matrix, 3)  # Fix floating-point issues
condensed_matrix = squareform(distance_matrix)

# Perform original hierarchical clustering
Z_original = sch.linkage(condensed_matrix, method="average")

# Bootstrapping
num_bootstraps = 100  # Number of bootstrap resamples
bootstrap_support = defaultdict(int)

for _ in range(num_bootstraps):
    # Resample genes with replacement
    resampled_indices = np.random.choice(len(genes), size=len(genes), replace=True)
    resampled_genes = [genes[i] for i in resampled_indices]
    
    # Subset distance matrix
    resampled_distances = distance_matrix[np.ix_(resampled_indices, resampled_indices)]
    
    # Compute linkage on bootstrap sample
    condensed_resampled = squareform(resampled_distances)
    Z_resampled = sch.linkage(condensed_resampled, method="average")
    
    # Convert to set of clusters
    for i in range(len(Z_resampled)):
        cluster = frozenset(resampled_genes[int(Z_resampled[i, 0]) : int(Z_resampled[i, 1]) + 1])
        bootstrap_support[cluster] += 1

# Compute bootstrap percentages
for cluster in bootstrap_support:
    bootstrap_support[cluster] = (bootstrap_support[cluster] / num_bootstraps) * 100

# Function to create Newick format with bootstrap support
def generate_newick(Z, bootstrap_support, genes):
    def recurse(node):
        if node < len(genes):  # Leaf node
            return genes[int(node)]
        else:  # Internal node
            left = recurse(Z[int(node - len(genes)), 0])
            right = recurse(Z[int(node - len(genes)), 1])
            support = bootstrap_support.get(frozenset([left, right]), 0)
            return f"({left},{right}){':' + str(round(support, 2)) if support > 0 else ''}"

    # Start the recursive generation of the Newick string
    return recurse(len(Z) + len(genes) - 1) + ';'

# Generate Newick tree with bootstrap support
newick_tree = generate_newick(Z_original, bootstrap_support, genes)

# Print the Newick tree with bootstrap values
print("Newick Tree with Bootstrap Confidence:")
print(newick_tree)



Newick Tree with Bootstrap Confidence:
((gene4,(gene5,gene6):11.0),(gene3,(gene1,gene2):15.0));
